<a href="https://colab.research.google.com/github/zakariaEnnahhal/Data-Science--Cheat-Sheet/blob/master/4HDATA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import pandas as pd
import logging
from tqdm import tqdm
import numpy as np
from tqdm import trange
from moviepy import *
from moviepy.editor import VideoFileClip
import moviepy.editor as mp
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram # Convert data to Spectogram
import librosa # https://librosa.org/doc/latest/index.html
import numpy as np
from pathlib import Path
import os
import librosa.display
import gc
import shutil
import glob
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.vision import *
from fastai.vision.all import *

from pathlib import Path
import torchaudio
from sklearn.model_selection import train_test_split

import os
import sys


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
from pathlib import Path

#download_path = Path.cwd()/'UrbanSound8K'

# Read metadata file
#metadata_file = download_path/'metadata'/'UrbanSound8K.csv'
df = pd.read_csv("/content/gdrive/MyDrive/DATASET E-R/annotation_final_4h.csv")
df.head()

,name,agreement,gender,duration
0,fm_01_angry_0,0,female,5.04
1,fm_01_angry_1,0,female,5.04
2,fm_01_angry_10,0,female,5.04
3,fm_01_angry_11,0,female,5.04
4,fm_01_angry_12,0,female,5.04


In [6]:
!pip install ffmpeg moviepy

In [7]:
data = []

for path in tqdm(Path("/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('_')[4]

    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "name": name,
            "path": path,
            "emotion": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

963it [00:30, 31.09it/s]


In [8]:
for path in tqdm(Path("/content/gdrive/MyDrive/DATA_Audio_4H/4h/Calm").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('_')[4]

    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "name": name,
            "path": path,
            "emotion": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

1052it [00:36, 29.08it/s]


In [9]:
for path in tqdm(Path("/content/gdrive/MyDrive/DATA_Audio_4H/4h/Happy").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('_')[4]

    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "name": name,
            "path": path,
            "emotion": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

604it [00:27, 22.30it/s]


In [10]:
for path in tqdm(Path("/content/gdrive/MyDrive/DATA_Audio_4H/4h/Sad").glob("**/*.wav")):
    name = str(path).split('/')[-1].split('.')[0]
    label = str(path).split('_')[4]

    try:
        # There are some broken files
        s = torchaudio.load(path)
        data.append({
            "name": name,
            "path": path,
            "emotion": label
        })
    except Exception as e:
        # print(str(path), e)
        pass

    # break

581it [00:16, 35.31it/s]


In [11]:
df = pd.DataFrame(data)
df.head()

,name,path,emotion
0,ml_2626_angry_1_t_15,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2626_angry_1_t_15.wav,angry
1,ml_2626_angry_1_t_16,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2626_angry_1_t_16.wav,angry
2,ml_2626_angry_1_t_17,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2626_angry_1_t_17.wav,angry
3,ml_2626_angry_2_t_0,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2626_angry_2_t_0.wav,angry
4,ml_2626_angry_2_t_1,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2626_angry_2_t_1.wav,angry


In [12]:
print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

df = df.sample(frac=1)
df = df.reset_index(drop=True)
df.head()

Step 0: 3200


  df = df.drop("status", 1)



Step 1: 3200


,name,path,emotion
0,fm_3651_angry_4,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/fm_3651_angry_4.wav,angry
1,ml_34_calm_0,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Calm/ml_34_calm_0.wav,calm
2,fm_26_happy_2_0,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Happy/fm_26_happy_2_0.wav,happy
3,fm_04_sad_3_1,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Sad/fm_04_sad_3_1.wav,sad
4,ml_2654_angry_2,/content/gdrive/MyDrive/DATA_Audio_4H/4h/Angry/ml_2654_angry_2.wav,angry


In [13]:
print("Labels: ", df["emotion"].unique())
print()
df.groupby("emotion").count()[["path"]]

Labels:  ['angry' 'calm' 'happy' 'sad']



,path
emotion,
angry,963
calm,1052
happy,604
sad,581


In [14]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["emotion"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
speech = speech[0].numpy().squeeze()
print(sr)
speech = librosa.resample(np.asarray(speech),orig_sr=sr, target_sr=16000)
ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 2952
      Label: happy

44100


In [15]:
save_path = "/content/gdrive/MyDrive/DATA_Audio_4H/4h"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["emotion"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(2560, 3)
(640, 3)


In [54]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

FileNotFoundError: [Errno 2] No such file or directory: './vocab.json'

In [52]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [53]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

NameError: name 'tokenizer' is not defined

In [16]:
!pip install datasets

In [29]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric


data_files = {
    "train": "/content/gdrive/MyDrive/DATA_Audio_4H/4h/train.csv",
    "validation": "/content/gdrive/MyDrive/DATA_Audio_4H/4h/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 2560
})
Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 640
})


In [30]:
input_column = "path"
output_column = "emotion"

In [32]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 4 classes: ['angry', 'calm', 'happy', 'sad']


In [33]:
train_dataset[0]

{'name': 'fm_1041_sad_4',
 'path': '/content/gdrive/MyDrive/DATA_Audio_4H/4h/Sad/fm_1041_sad_4.wav',
 'emotion': 'sad'}

In [37]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["Emotion"] = batch["emotion"]
    return batch

In [ ]:
train_dataset1 = train_dataset.map(speech_file_to_array_fn, remove_columns=train_dataset.column_names)

In [38]:
eval_dataset1 = eval_dataset.map(speech_file_to_array_fn, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/640 [00:00<?, ? examples/s]

In [39]:
train_dataset1

Dataset({
    features: ['speech', 'sampling_rate', 'Emotion'],
    num_rows: 2560
})

In [40]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), orig_sr=48_000, target_sr=16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [41]:
train_dataset1 = train_dataset1.map(resample, num_proc=4)

Map (num_proc=4):   0%|          | 0/2560 [00:00<?, ? examples/s]

In [42]:
eval_dataset1 = eval_dataset1.map(resample, num_proc=4)

Map (num_proc=4):   0%|          | 0/640 [00:00<?, ? examples/s]

In [46]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset1))

ipd.Audio(data=np.asarray(train_dataset1[rand_int]["speech"]), autoplay=True, rate=16000)

In [48]:
rand_int = random.randint(0, len(train_dataset1))

print("Emotion:", train_dataset1[rand_int]["Emotion"])
print("Input array shape:", np.asarray(train_dataset1[rand_int]["speech"]).shape)
print("Sampling rate:", train_dataset1[rand_int]["sampling_rate"])

Emotion: calm
Input array shape: (74088,)
Sampling rate: 16000


In [56]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None



In [57]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )


In [65]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [66]:
Data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [61]:
from transformers import AutoConfig, Wav2Vec2Processor

In [62]:
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"


In [63]:

# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

  warnings.warn(



In [64]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


In [67]:
is_regression = False

In [68]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [69]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

  return self.fget.__get__(instance, owner)()

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at lighteternal/wav2vec2-large-xlsr-53-greek and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
model.freeze_feature_extractor()

In [74]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.5 MB/s eta 0:00:00


In [1]:
from transformers import TrainingArguments

training_args = TrainingArguments(
      output_dir="/content/wav2vec2-xlsr-greek-speech-emotion-recognition",
   #output_dir="/content/gdrive/MyDrive/DATA_Audio_4H"
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
)

In [2]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [3]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

NameError: name 'model' is not defined

In [ ]:
# collator
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        # d_type = torch.long if isinstance(label_features[0], int) else torch.float
        d_type = torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor

In [ ]:
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
    problem_type="regression"
)
setattr(config, 'pooling_mode', pooling_mode)

In [ ]:
config.problem_type

'regression'

In [ ]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label for label in examples[output_column]] # Do any preprocessing on your float/integer data

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
train_dataset = train_dataset.select(range(100))
eval_dataset = eval_dataset.select(range(100))

In [ ]:
train_dataset

Dataset({
    features: ['name', 'path', 'emotion'],
    num_rows: 100
})

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=1,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=1,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['emotion']}")
print(f"Type of label: {type(train_dataset[idx]['labels'])}")

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Training attention_mask: [1, 1]
Training labels: sad - sad
Type of label: <class 'str'>


In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor

In [ ]:
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

  warnings.warn(



In [ ]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]
    result = processor(speech_list, padding=True, sampling_rate=target_sampling_rate)
    #result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)


Map (num_proc=4):   0%|          | 0/2560 [00:00<?, ? examples/s]

ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=4,
    padding=True
)

TypeError: Dataset.map() got an unexpected keyword argument 'padding'

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor, Wav2Vec2FeatureExtractor

In [ ]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
target_sampling_rate = feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

The target sampling rate: 16000


In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
# Remove this part
max_samples = 100
train_dataset = train_dataset.select(range(max_samples))
eval_dataset = eval_dataset.select(range(max_samples))

In [ ]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)

  StockPickler.save(self, obj, save_persistent_id)

  StockPickler.save(self, obj, save_persistent_id)

  StockPickler.save(self, obj, save_persistent_id)

  StockPickler.save(self, obj, save_persistent_id)



Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
target_sampling_rate = feature_extractor.sampling_rate
print(feature_extractor)


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



In [ ]:
model_name_or_path = "facebook/wav2vec2-base-100k-voxpopuli"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path,)
target_sampling_rate = feature_extractor.sampling_rate
print(feature_extractor)

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}



In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    print(speech_list)
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]
    print(type(speech_list))
    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)

    result["labels"] = list(target_list)

    return result

# Remove this part
max_samples = 100
train_dataset = train_dataset.select(range(max_samples))
eval_dataset = eval_dataset.select(range(max_samples))

train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)

NameError: name 'train_dataset' is not defined

In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    print(speech_list)
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]
    print(type(speech_list))
    result = feature_extractor(speech_list, sampling_rate=target_sampling_rate)

    result["labels"] = list(target_list)

    return result

# Remove this part
max_samples = 100
train_dataset = train_dataset.select(range(max_samples))
eval_dataset = eval_dataset.select(range(max_samples))

train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=10,
    batched=True,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

[array([[ 0.02234694,  0.03066144,  0.01269014, ...,  0.03333434,
        -0.0499514 , -0.0793226 ],
       [ 0.02234694,  0.03066144,  0.01269014, ...,  0.03333434,
        -0.0499514 , -0.0793226 ]], dtype=float32), array([[ 0.00194434,  0.00425758,  0.00485958, ..., -0.02450845,
        -0.02203635, -0.03469632],
       [ 0.00039563,  0.00123737,  0.00214268, ...,  0.01904412,
         0.01919332, -0.00557659]], dtype=float32), array([[-0.01296568, -0.03672482, -0.05676255, ...,  0.        ,
         0.        ,  0.        ],
       [-0.01296568, -0.03672482, -0.05676255, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32), array([[-0.02966   , -0.0469931 , -0.05225345, ...,  0.01876768,
         0.01435301,  0.00814889],
       [-0.02966   , -0.0469931 , -0.05225345, ...,  0.01876768,
         0.01435301,  0.00814889]], dtype=float32), array([[ 2.5096232e-01,  2.8307691e-01,  2.1410674e-01, ...,
         9.2833747e-05,  6.3106299e-06, -2.6625095e-04],
       [ 2.50

ValueError: Unable to create tensor, you should probably activate padding with 'padding=True' to have batched tensors with the same length.

In [ ]:
import librosa as sf

In [ ]:

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["audio"] = speech_array
    return batch

updated_dataset =train_dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

AttributeError: No librosa attribute read

In [ ]:
train_dataset.set_format("numpy", output_all_columns=True)
train_dataset["test"][0]

NameError: name 'train_dataset' is not defined